# Exercise 3: Compute NMR Shielding Constants and Magnetizibility for a Transition Metal Complex

Below you will find links to all all the relevant pages of the MRChem documentation. Please refer to these if you need more details on how to set up the calculations.

[<img src="https://img.shields.io/badge/Documentation-Main_Page-orange.svg?logo=LOGO">](https://mrchem.readthedocs.io/en/latest/)
[<img src="https://img.shields.io/badge/I_Need_Help-Installation-teal.svg?logo=LOGO">](https://mrchem.readthedocs.io/en/latest/installation.html)
[<img src="https://img.shields.io/badge/I_Need_Help-How_To_Run-green.svg?logo=LOGO">](https://mrchem.readthedocs.io/en/latest/users/running.html)
[<img src="https://img.shields.io/badge/I_Need_Help-Input_Quick_Guide-blue.svg?logo=LOGO">](https://mrchem.readthedocs.io/en/latest/users/user_inp.html)
[<img src="https://img.shields.io/badge/I_Need_Help-Input_Reference-purple.svg?logo=LOGO">](https://mrchem.readthedocs.io/en/latest/users/user_ref.html)
[<img src="https://img.shields.io/badge/I_Need_Help-JSON_Output-red.svg?logo=LOGO">](https://mrchem.readthedocs.io/en/latest/users/program_json.html)

## Introduction
In this exercise you will use the MRChem code to compute NMR shielding constants and magnetizibilities for a Ni(CO)_3 complex bound to an N-Heterocyclic Carbene (NHC) ligand. 
NHCs are commonly used in organometallic chemistry, and are often involved in catalytic cycles.
They contain a zero-valent carbon atom that acts as a very strong sigma-donor, which often coordinates to a transition metal.
Due to the bulky nature of many NHCs, they are used to help stabilize reactive or exotic chemical species.

The NHC used here is depicted in the image below. One usually tunes the NHC by substituing the methyl groups with larger and more bulky side groups.
______

## How to run the calculation
Due to the size of the system, we need to run this calculation on one of the national HPC clusters. The instructions here are specific to `Saga`.

We need a submit script where we set up the calculation, since we are submitting to the `SLURM` queueing manager. We will be using MRChem in a hybrid OpenMP+MPI parallelization mode, which ensures very good scaling on HPC clusters. 

We provide a convenience script that you can copy to the cluster (`runMRChem.sh`), to use for setting up the SLURM variables, loading MRChem, and copying files to/from the scratch directory.
Here is the usage:

```
Usage: ./runMRChem.sh [ -h ] [ -i JOBNAME ] [ -t NTASKS ] [ -c NCPUS ] [ -T TIMELIMIT ] [ -m MEMORY ] [ -x ]

Optional arguments:
-h                : Show this message and exit 0
-i <JOBNAME>      : Input file w.o. extension (.inp assumed)  default: jobname
-t <NTASKS>       : Number of MPI processes                   default: 8
-c <NCPUS>        : Number of OMP threads                     default: 8
-T <TIMELIMIT>    : Wall time limit in SLURM format           default: 10:00:00
-m <MEMORY>       : Total memory in GB                        default: 60
-x                : Submit job to queue                       default: false
```

**Note**: You may have to make the script executable:

```console
user@saga:~$ chmod +x runMRChem.sh
```

The file `runMRChem.sh` generates a job file `jobname.job` like this (using default values):

```bash
#!/bin/bash

#SBATCH --account=nn4654k
#SBATCH --mail-type=None
#SBATCH --job-name=jobname
#SBATCH --output=jobname.log
#SBATCH --error=jobname.err
#SBATCH --time=10:00:00
#SBATCH --mem=60GB
#SBATCH --ntasks=8
#SBATCH --cpus-per-task=8

module purge
module load MRChem/toolchain-version

cp jobname.inp 
cd 
export OMP_NUM_THREADS=8
mrchem --launcher 'srun -n 8' jobname.inp

savefile jobname.out
savefile jobname.json
exit 0
```

which you can submit to the queue with this command

```console
user@saga:~$ sbatch jobname.job
```

You can also let the script submit your job automatically by adding the `-x` flag (but this only makes sense if you are running the script on the remote cluster).
If your jobfile already exists, you will be prompted to confirm the overwrite.

Example usage:

```console
user@saga:~$ ./runMRChem.sh -i Ni_NHC -t 25 -T '30:00:00' -m 160GB -x
```
___________

## The MRChem input file
Below is an outline of an MRChem input for an NMR properties calculation at the MW3 precision level.
Try to use the documentation pages to find the relevant keywords for setting up the calculation.
If you need additional help, don't hesitate to ask :)

```
world_prec = 1.0e-3
world_unit = angstrom

Molecule {
  $coords
  ...
  $end
}

WaveFunction {
}

SCF {
}

Properties {
}

Response {
}

```
______

## Computational details
* The GTO results were computed with the `PBE` functional, and so you should also use `PBE` if the results are to comparable.
* An optimized geometry of the TM complex is supplied in `./geometries/Ni_NHC.xyz`.
* Use a KAIN SCF and response accelerator history of 15.
* Make sure the SCF and response `max_iter` options are large enough (100 should be plenty).
* As starting guess, `sad_tz` at MW4 precision works well.

> **Note**
> Input blocks are case sensitive, while keywords inside blocks are not
_____

## Instructions
 1. Generate the input files. You can either do this on the cluster, or write your own Python functions here and then copy to the cluster.
 
 2. Submit the calculations as described above. Check in on the job regularly to see if it runs as expected.
 
 3. You are encouraged to take a look at the output file (`jobname.out`) to look at the information printed here, and also to check out the JSON output file (`jobname.json`) to see the same output in a very scripting friendly format.
 
 4. Extract the relevant information from either the `jobname.out` file or the `jobname.json` file, and do something interesting with it???
  _______

## Convenience scripts
A class `MRChemOutput` is provided in the `./utils/functions.py` file to aid in fetching relevant properties from the JSON output file. In addition, it has methods for plotting the SCF convergence.

(You can of course also open the output files directly in the terminal and extract the data manually if you prefer.)

In [1]:
from utils.functions import MRChemOutput

help(MRChemOutput)

Help on class MRChemOutput in module utils.functions:

class MRChemOutput(builtins.object)
 |  MRChemOutput(jsonfile)
 |  
 |  Convenience class for accessing data from an MRChem calculation.
 |  
 |  You can access the JSON data via the data attribute.
 |  For example, to get the final SCF total energy:
 |  
 |  
 |  []: calc = MRChemOutput('jobname.json')
 |      E_tot = calc.data['output']['properties']['scf_energy']['E_tot']
 |  
 |  Additionally, the JSON is loaded as a nested SimpleNamespace
 |  (stored in the ns attribute),
 |  and you can quickly navigate the dict by 'dotting' through
 |  the keys. Tab completion should work for this in the Jupyter
 |  environment:
 |  
 |  []: E_tot = calc.ns.output.properties.scf_energy.E_tot
 |  
 |  Known bug
 |  Some keys are not valid Python variable names,
 |  and you cannot access the levels below these keys by dotting.
 |  Workaround: use __.getattribute__(key) to access such keys.
 |  
 |  A few methods are also provided that serve as